In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#import other libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#pandas dataframe configuration
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format

In [3]:
DATA_PATH = '../input/store-sales-time-series-forecasting/'

In [4]:
for f in os. listdir(DATA_PATH):
    print(f)

In [5]:
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

df_stores = pd.read_csv(os.path.join(DATA_PATH, 'stores.csv'))
df_transactions = pd.read_csv(os.path.join(DATA_PATH, 'transactions.csv')).sort_values(['store_nbr', 'date'])
df_oil = pd.read_csv(os.path.join(DATA_PATH, 'oil.csv'))
df_holidays = pd.read_csv(os.path.join(DATA_PATH, 'holidays_events.csv'))

In [6]:
#Datetime
df_train["date"] = pd.to_datetime(df_train['date'])
df_test["date"] = pd.to_datetime(df_test['date'])
df_transactions["date"] = pd.to_datetime(df_transactions['date'])

In [7]:
df_train.sales = df_train.sales.astype('float32')
df_stores.cluster = df_stores.cluster.astype('int8')

In [8]:
type(df_train['date'][0])

In [9]:
df_train

In [10]:
average_onpromotion = df_train.groupby('date').mean()['onpromotion']

In [11]:
average_onpromotion.plot(style='.', figsize=(20,10))

In [12]:
avg_train = average_onpromotion.copy()

In [13]:
moving_avg = avg_train.rolling(window=365, center=True, min_periods=183).mean()

In [14]:
avg_train.plot(style='x', figsize=(20,10));
moving_avg.plot()

In [15]:
df_train_temp = df_train.groupby(['store_nbr', 'onpromotion', 'date']).sales.sum().reset_index()

In [16]:
print(df_stores.columns)
print(df_train.columns)
print(df_transactions.columns)
print(df_holidays.columns)

In [17]:
df_aux_merged = pd.merge(df_train_temp, df_stores, how='left')

In [18]:
df_aux_merged

In [19]:
pearson_corr = df_aux_merged.corr('pearson')['sales'].loc['cluster']
spearman_corr = df_aux_merged.corr('spearman')['sales'].loc['cluster']

In [20]:
print(f'pearson_corr = {pearson_corr:.4f} and spearman corr = {spearman_corr:.4f}')

In [21]:
df_sorted_stores = df_aux_merged.sort_values(['cluster', 'date'])

In [22]:
px.line(df_sorted_stores, x='date', y='onpromotion', color='cluster', title='Onpromotion by date and cluster')

In [23]:
aux_a = df_aux_merged.copy()
aux_a['year'] = aux_a.date.dt.year
aux_a['month'] = aux_a['date'].dt.month

In [24]:
px.box(aux_a, x='year', y='onpromotion', color='month', title='Box plot onpromotion')

In [25]:
aux_a = df_aux_merged.set_index('date').resample('M').onpromotion.mean().reset_index()

In [26]:
aux_a['year'] = aux_a.date.dt.year

In [27]:
px.line(aux_a, x='date', y='onpromotion', color='year', title='Monthly transactions')